In [ ]:
#|hide
from ntlights_damage_assessment.core import *

# Nightlights for Damage Assessment
> Using nightlights satellite imagery for typhoon damage assessment

## Install

`pip install git+https://github.com/butchland/ntlights-damage-assessment.git`

## How to use

* Download the nightlights satellite imagery for the period before and after the disaster
* Download the geospatial admin boundaries for the areas concerned
* Download and scrape the damage assessment situationers as ground truth
* Preprocess the situationrers to extract the locations and damage assessment
* Wrangle and link the damage assessment locations to the admin boundaries
* Extract the radiance data from the nightlights satellite
* Analyze the patterns for radiance data for highly damaged areas vs undamaged or low damage areas
* Write up conclusions


In [ ]:
say_hello('butch')

'hello, butch'